<a href="https://colab.research.google.com/github/junior0428/GEE/blob/main/02_EDGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import ee
#ee.Authenticate()
ee.Initialize()

In [28]:

#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [9]:
image1=ee.Image(1)
#print(image1)
image1.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'}],
 'type': 'Image'}

In [ ]:
#concatenar bandas 
image2=ee.Image(20)
image3=ee.Image.cat([image1, image2])
image3.getInfo()

In [ ]:
#otra forma de concatenar bandas 
imag=ee.Image([10,30])
imag.getInfo()

In [14]:
#Otra forma de concatenar bandas que es mas utilizado
image4=image1.addBands(imag)
image4.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 30,
    'min': 30,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1_1'}],
 'type': 'Image'}

In [18]:
#rename el ID de cada imagen
image4.bandNames().getInfo()
image5=image4.rename(['img_01', 'img_02', 'img_3'])
image5.bandNames().getInfo()
image5.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'img_01'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'img_02'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 30,
    'min': 30,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'img_3'}],
 'type': 'Image'}

In [20]:
#seleccionar las bandas de interes
image5.select(['img_3']).bandNames().getInfo()

['img_3']

##**Indices espectrales**

In [29]:
img=ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_008066_20200610')
#img.getInfo()
centr_img=img.geometry().centroid().coordinates().getInfo()# optenemos las coordenadas del centro
img.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [33]:
mydicc_viz={'landsat8': img.getMapId({'bands':['B7', 'B5', 'B3'], 'max':0.5, 'min':0})}
mapdisplay(centr_img, mydicc_viz)

##**Normalized difference vegetation index**

In [42]:
nir=img.select(['B5'])
red=img.select(['B4'])
ndvi=nir.subtract(red).divide(nir.add(red))
ndvi02=nir.subtract(red).divide(nir.add(red))
ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
mydicc = {
    'ndvi-method01': ndvi.getMapId({'min':0,'max':0.8,'palette':ndvi_palette}),
    'ndvi-method02': ndvi02.getMapId({'min':-0.1,'max':0.8,'palette':ndvi_palette}),
}
mapdisplay(centr_img, mydicc)